# Initialization

In [1]:
import pandas as pd
import re
import numpy as np
import urllib
from bs4 import BeautifulSoup as Soup
import os
from pathlib import Path

In [363]:
speakers = pd.DataFrame(columns=["speaker_id", 
                                "first_name", 
                                "last_name",
                                "chamber",
                                "type",
                                "party", 
                                "state", 
                                "district",
                                "bio_guide_id",
                                "congress_id"])

In [364]:
speakers

,speaker_id,first_name,last_name,chamber,type,party,state,district,bio_guide_id,congress_id


In [365]:
speeches = pd.DataFrame(columns=["speech_id",
                                 "last_name",
                                 "speaker_id",
                                 "proceeding_id", 
                                 "topic_id", 
                                 "word_count", 
                                 "speech_text",
                                 "file_name",
                                 "mods_file"])

In [366]:
speeches

,speech_id,last_name,speaker_id,proceeding_id,topic_id,word_count,speech_text,file_name,mods_file


# Master Mods Parsing

In [2]:
def get_all_extensions(file):
    handler = open(file).read()
    soup = Soup(handler, "lxml")
    return soup.find_all("extension")

In [3]:
def get_cong_member_tag(cong_member_extension):
    cong_member_tag = cong_member_extension.find("congmember")
    return cong_member_tag

In [4]:
def get_party(cong_member_tag):
    try:
        return cong_member_tag.attrs['party']
    except:
        return 'N/A'

def get_type(cong_member_tag):
    try:
        return cong_member_tag.attrs['type']
    except:
        return 'N/A'

def get_authority_id(cong_member_tag):
    try:
        return cong_member_tag.attrs['authorityid']
    except:
        print("auth id not found")
        return None

def get_bioguide_id(cong_member_tag):
    try:
        return cong_member_tag.attrs['bioguideid']
    except:
        return 'N/A'

def get_state(cong_member_tag):
    try:
        return cong_member_tag.attrs['state']
    except:
        return 'N/A'

def get_congress_id(cong_member_tag):
    try:
        return cong_member_tag.attrs['congress']
    except:
        return 'N/A'

def get_chamber(cong_member_tag):
    chambers = {'S': 'SENATE', 'H': 'HOUSE'}
    try:
        letter = cong_member_tag.attrs['chamber']
        return chambers[letter]
    except:
        return 'N/A'


In [5]:
def get_district(cong_member_extension):
    district_tag = cong_member_extension.find("district")
    if district_tag == None:
        return None
    return district_tag.string

In [6]:
def get_first_name(cong_member_tag):
    name_tag_arr = cong_member_tag.select("name[type='authority-fnf']")
    if name_tag_arr == []:
        print("no first_name")
        print(cong_member_tag)
        return 'N/A'
    name_tag = name_tag_arr[0]
    first_name = name_tag.text.split()[0]
    return first_name

In [7]:
def get_last_name(cong_member_tag):
    name_tag_arr = cong_member_tag.select("name[type='authority-lnf']")
    if name_tag_arr == []:
        print("no first_name")
        print(cong_member_tag)
        return 'N/A'
    name_tag = name_tag_arr[0]
    full_name = name_tag.string
    return re.match("[^,]*", full_name).group(0).upper()

In [13]:
filenames = os.listdir("/Users/halliday/projects/searchlight/parsing/mastermods")
filenames.remove('.DS_Store')
for filename in filenames:
    extensions = get_all_extensions("/Users/halliday/projects/searchlight/parsing/mastermods/" + filename)
    print("            " + filename)
    for extension in extensions:
        cong_member_tag = get_cong_member_tag(extension)
        if cong_member_tag == None:
            continue
#         if filename == 'mods112.xml':
#             print(cong_member_tag)
        authority_id = get_authority_id(cong_member_tag)
        if speakers[speakers['speaker_id'] == authority_id]['speaker_id'].count() > 0 or authority_id == "" or authority_id == None:
            continue
        cong_member_dict = {'speaker_id': authority_id, 
                            'first_name': get_first_name(cong_member_tag),
                            'last_name': get_last_name(cong_member_tag),
                            'chamber': get_chamber(cong_member_tag),
                            'type': get_type(cong_member_tag),
                            'party': get_party(cong_member_tag),
                            'state': get_state(cong_member_tag),
                            'district': get_district(extension),
                            'bio_guide_id': get_bioguide_id(cong_member_tag),
                            'congress_id': get_congress_id(cong_member_tag)}
        speakers = speakers.append(cong_member_dict, ignore_index=True)

            mods113.xml


NameError: name 'speakers' is not defined

In [ ]:
speakers.head()

In [319]:
speakers[speakers['congress_id'] == 'N/A']

,speaker_id,first_name,last_name,chamber,type,party,state,district,bio_guide_id,congress_id


In [320]:
speakers = speakers.sort_values('last_name')

In [321]:
speakers.to_csv('masterspeakers.csv', index=False)

# Speeches Parsing

In [367]:
speakersDir = Path('/Users/halliday/projects/searchlight/parsing')
speakers = pd.read_csv(speakersDir/'updatedspeakers.csv')
speakers.head()

,speaker_id,first_name,last_name,chamber,type,party,state,district,bio_guide_id,congress_id
0,2,Neil,ABERCROMBIE,HOUSE,REPRESENTATIVE,D,HI,1.0,A000014,107
1,1269,Spencer,ABRAHAM,SENATE,SENATOR,R,MI,NaN,A000355,106
2,2244,Ralph,ABRAHAM,HOUSE,REPRESENTATIVE,R,LA,5.0,A000374,114
3,4,Gary,ACKERMAN,HOUSE,REPRESENTATIVE,D,NY,5.0,A000022,107
4,2006,Sandy,ADAMS,HOUSE,REPRESENTATIVE,R,FL,24.0,A000366,112


In [8]:
def remove_space(regex):
    return regex.group().replace(' ', '')

In [9]:
def sep_speech(filepath):
    parse_file = ''
    with open(filepath) as file:
        for line in file:
            parse_file += line
    parse_file = parse_file.replace('\n', '')
    parse_file = re.sub('Mr. [A-Z][a-z]', remove_space, parse_file)
    
    split = re.split(r'Mr. |Ms. |Mrs. ', parse_file)
    split.pop(0)
    name_and_speech = []
    for i in np.arange(len(split)):
        try:
            lastname = re.match('[A-Z]*\. ', split[i]).group(0)[:-2]
            name_and_speech += [lastname]
            value = re.sub('[A-Z]\w*\. ', '', split[i])
            name_and_speech += [value]
        except:
            continue
    return name_and_speech

In [10]:
def sep_date_from_file(file):
    abcdef = re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}', file)
    return re.split('-', abcdef[0])

In [11]:
def is_int(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [12]:
def find_title(file_path):
    parse_file = ''
    with open(file_path) as file:
        for line in file:
            parse_file += line
    parse_file = parse_file.replace('Mr. President', 'MrPresident')
    titles = re.findall('[A-Z \'-]+[A-Z0-9-,\. ]*[Continued]*\\n', parse_file)
    if is_int(titles[0].strip().replace('\n', '')):
        return titles[1].strip()
    else:
        return titles[0].strip()

In [13]:
def fix_surname_typos(name):
    if name == 'SOUZZI':
        return 'SUOZZI'
    if name == 'VANHOLLEN':
        return 'VAN HOLLEN'
    if name == 'FISHCER':
        return 'FISCHER'
    return name

# Local Mods Parsing

In [14]:
def get_cong_member_tag_from_mods(last_name, mods_file_path):
    try:
        handler = open(mods_file_path).read()
    except:
        return None
    soup = Soup(handler, "lxml")
    cong_member_tags = soup.find_all("congmember")
    matched_cong_member_tag = None
    for i in range(len(cong_member_tags)):
        curr_last_name = get_last_name(cong_member_tags[i])
        if curr_last_name == last_name:
            matched_cong_member_tag = cong_member_tags[i]
            break
    return matched_cong_member_tag

In [15]:
def get_cong_member_info(last_name, mods_file_path):
    matched_cong_member_tag = get_cong_member_tag_from_mods(last_name, mods_file_path)
    if matched_cong_member_tag == None:
        return {'speaker_id': None, 
                'first_name': 'N/A',
                'last_name': last_name,
                'type': 'N/A',
                'chamber': 'N/A',
                'party': 'N/A',
                'state': 'N/A',
                'district': None,
                'bio_guide_id': 'N/A',
                'congress_id': 'N/A'}
    else:
        return {'speaker_id': get_authority_id(matched_cong_member_tag), 
                'first_name': get_first_name(matched_cong_member_tag),
                'last_name': last_name,
                'chamber': get_chamber(matched_cong_member_tag),
                'type': get_type(matched_cong_member_tag),
                'party': get_party(matched_cong_member_tag),
                'state': get_state(matched_cong_member_tag),
                'district': None,
                'bio_guide_id': get_bioguide_id(matched_cong_member_tag),
                'congress_id': get_congress_id(matched_cong_member_tag)}

In [16]:
def get_authority_id_from_mods(last_name, mods_file_path):
    matched_cong_member_tag = get_cong_member_tag_from_mods(last_name, mods_file_path)
    if matched_cong_member_tag == None:
        return 99999999999999
    authority_id = get_authority_id(matched_cong_member_tag)
    if authority_id == None:
        return 99999999999999
    else:
        return authority_id

In [17]:
def get_bill_context(bill_tag):
    try:
        return bill_tag.attrs['context']
    except:
        return 'N/A'
def get_bill_congress(bill_tag):
    try:
        return bill_tag.attrs['congress']
    except:
        return 'N/A'
def get_bill_number(bill_tag):
    try:
        return bill_tag.attrs['number']
    except:
        return 'N/A'
def get_bill_type(bill_tag):
    try:
        return bill_tag.attrs['type']
    except:
        return 'N/A'

In [21]:
def populate_speeches(count, folder, next_index):
    
    speakersDir = Path('/Users/halliday/projects/searchlight/parsing')
    speakers = pd.read_csv(speakersDir/'updatedspeakers.csv')
    
    speeches = pd.DataFrame(columns=["speech_id",
                                 "last_name",
                                 "speaker_id",
                                 "proceeding_id", 
                                 "topic_id", 
                                 "word_count", 
                                 "speech_text",
                                 "file_name",
                                 "mods_file"])
    
    bills = pd.DataFrame(columns=["mods_file",
                              "congress_id",
                              "context",
                              "bill_number",
                              "bill_type"])

    #collects speaker-speech pairs from text files
    def collect_pairs(folder):
        nonlocal speeches
        nonlocal next_index
        speech_count = next_index
        folder_path = "/Users/halliday/projects/searchlight/parsing/" + folder
        list_of_files = os.listdir(folder_path)
        for file in list_of_files:
            if file.endswith(".txt"):
                print(file)
                if file == 'CREC-2018-03-22-pt1-PgH1769-2.txt':
                    continue
                if file == 'CREC-2017-09-06-pt1-PgH6695.txt':
                    continue
                file_path = "/Users/halliday/projects/searchlight/parsing/" + folder + "/" + file
                mods_file = file.replace('.txt', '.xml')
                separated = sep_speech(file_path)
                i = 0
                while i < len(separated):
                    separated_surname = fix_surname_typos(separated[i])
                    text = separated[i+1]
                    text = text.replace('MrPresident', 'Mr. President')
                    if len(text) > 30:
                        row = {"speech_id": speech_count,
                               "last_name": separated_surname,
                               "speaker_id": 99999999999999,
                               "proceeding_id": "proceeding_id", 
                               "topic_id": "topic_id",
                               "word_count": len(text.split()), 
                               "speech_text": text,
                               "file_name": file,
                               "mods_file": mods_file}
                        speech_count += 1
                        speeches = speeches.append(row, ignore_index=True)     
                    i += 2
                print('finished with file ', speech_count)
    
    collect_pairs(folder)
        
    def get_speaker_id(last_name, mods_file_path):
        nonlocal speakers
        possible_speakers = speakers[speakers['last_name'] == last_name]
        if possible_speakers.shape[0] == 0:
            new_speaker = get_cong_member_info(last_name, mods_file_path)
            speakers = speakers.append(new_speaker, ignore_index=True)
            print(speakers.shape[0])
            speakers = speakers.sort_values('last_name')
            speakers.to_csv('updatedspeakers.csv', index=False)
            print("wrote in new speaker")
            print(new_speaker)
            return new_speaker['speaker_id']
        elif possible_speakers.shape[0] == 1:
            print('used existing row')
            return possible_speakers.iloc[0]['speaker_id']
        else:
            mods_speaker_id = get_authority_id_from_mods(last_name, mods_file_path)
            if int(mods_speaker_id) > 100000:
                print("speaker not found in mods: " + last_name)
                return None
            matched_speaker = possible_speakers[possible_speakers['speaker_id'] == int(mods_speaker_id)]
            if matched_speaker.shape[0] == 1:
                print("speaker matched successfully, used existing row")
                return matched_speaker.iloc[0]['speaker_id']
            elif matched_speaker.shape[0] == 0:
                new_speaker = get_cong_member_info(last_name, mods_file_path)
                speakers = speakers.append(new_speaker, ignore_index=True)
                print("wrote in new speaker")
                print(new_speaker)
                speakers = speakers.sort_values('last_name')
                speakers.to_csv('updatedspeakers.csv', index=False)
                return new_speaker['speaker_id']
    
    def get_bills(mods_file_path, mods_file):
        nonlocal bills
        
        file_exists = bills[bills['mods_file'] == mods_file]
        if file_exists.shape[0] > 0:
            return
        
        try:
            handler = open(mods_file_path).read()
        except:
            return
        soup = Soup(handler, "lxml")
        bill_tags = soup.find_all("bill")

        if len(bill_tags) == 0:
            return
        else:
            for bill_tag in bill_tags:
                new_bill_row = {"mods_file": mods_file,
                               "congress_id": get_bill_congress(bill_tag),
                               "context": get_bill_context(bill_tag),
                               "bill_number": get_bill_number(bill_tag),
                               "bill_type": get_bill_type(bill_tag)}
                bills = bills.append(new_bill_row, ignore_index=True)        
    
    #initialize new columns
    speeches['proceeding_title'], speeches['year'], speeches['month'], speeches['day'] = "", 0, 0, 0
    
    #collect speaker_ids, titles, dates, and bills
    for i in range(speeches.shape[0]):
        
        curr_row = speeches.iloc[i]
        last_name = curr_row['last_name']
        mods_file_path = "/Users/halliday/projects/searchlight/parsing/" + folder + "/" + curr_row['mods_file']
        text_file_path = "/Users/halliday/projects/searchlight/parsing/" + folder + "/" + curr_row['file_name']
        
        #collect speaker_ids, titles, and dates
        speaker_id = get_speaker_id(last_name, mods_file_path)
        title = find_title(text_file_path)
        year, month, day = sep_date_from_file(curr_row['file_name'])
        
        speeches.loc[i, "speaker_id"] = speaker_id
        speeches.loc[i, "proceeding_title"] = title
        speeches.loc[i, "year"] = int(year)
        speeches.loc[i, "month"] = int(month)
        speeches.loc[i, "day"] = int(day)
        
        #populate bills table if necessary
        get_bills(mods_file_path, curr_row['mods_file'])
    
    speeches = speeches.sort_values('speech_id')
    bills = bills.sort_values('mods_file')
    
    speeches.to_csv('./parsing_results/' + 'speeches_' + str(count) + ".csv", index=False)
    bills.to_csv('./parsing_results/' + 'bills_' + str(count) + ".csv", index=False)
    
    print("                " + "saved " + "speeches_" + str(count))
    print("                " + "saved " + "bills_" + str(count))
    
    return max(speeches['speech_id'])
    

In [22]:
count = 1
folders = os.listdir("./scraped_folders")
print(folders)
next_index = 0
for folder in folders:
    next_index = populate_speeches(count, "scraped_folders/" + folder, next_index)
    next_index += 1
    count += 1

['scrape2']
CREC-2017-05-17-pt1-PgH4312-2.txt
finished with file  0
CREC-2017-05-17-pt1-PgH4236-6.txt
finished with file  0
CREC-2017-06-15-pt1-PgH4923-2.txt
finished with file  0
CREC-2017-04-07-pt1-PgS2463-2.txt
finished with file  0
CREC-2017-05-16-pt1-PgH4203-9.txt
finished with file  0
CREC-2017-07-18-pt1-PgS4030.txt
finished with file  0
CREC-2017-07-12-pt1-PgH5439.txt
finished with file  1
CREC-2017-04-24-pt1-PgS2475-3.txt
finished with file  1
CREC-2017-07-13-pt1-PgS3992-2.txt
finished with file  2
CREC-2017-05-04-pt1-PgH4180-2.txt
finished with file  2
CREC-2017-06-06-pt1-PgH4642-6.txt
finished with file  3
CREC-2017-05-18-pt1-PgH4345-2.txt
finished with file  4
CREC-2017-05-01-pt1-PgS2648.txt
finished with file  5
CREC-2017-04-27-pt1-PgS2612-4.txt
finished with file  6
CREC-2017-06-27-pt1-PgH5183-2.txt
finished with file  6
CREC-2017-07-31-pt1-PgS4629-2.txt
finished with file  6
CREC-2017-05-02-pt1-PgH3068-10.txt
finished with file  6
CREC-2017-05-19-pt1-PgH4390-8.txt
finishe

finished with file  348
CREC-2017-06-14-pt1-PgH4918-3.txt
finished with file  348
CREC-2017-05-18-pt1-PgH4358-6.txt
finished with file  348
CREC-2017-06-13-pt1-PgH4900-2.txt
finished with file  348
CREC-2017-07-11-pt1-PgH5433-5.txt
finished with file  348
CREC-2017-06-12-pt1-PgH4854-9.txt
finished with file  348
CREC-2017-06-12-pt1-PgS3399.txt
finished with file  348
CREC-2017-07-20-pt1-PgH6146-8.txt
finished with file  348
CREC-2017-05-25-pt1-PgS3224.txt
finished with file  348
CREC-2017-07-24-pt1-PgS4135-3.txt
finished with file  348
CREC-2017-07-24-pt1-PgH6213-5.txt
finished with file  348
CREC-2017-07-17-pt1-PgH5903-5.txt
finished with file  349
CREC-2017-05-19-pt1-PgH4380.txt
finished with file  350
CREC-2017-05-24-pt1-PgH4518-4.txt
finished with file  351
CREC-2017-05-25-pt1-PgS3225-5.txt
finished with file  351
CREC-2017-04-26-pt1-PgS2549.txt
finished with file  352
CREC-2017-05-03-pt1-PgS2721.txt
finished with file  352
CREC-2017-05-01-pt1-PgS2649.txt
finished with file  353
CR

finished with file  473
CREC-2017-06-28-pt1-PgH5296-7.txt
finished with file  473
CREC-2017-04-26-pt1-PgH2859.txt
finished with file  474
CREC-2017-04-27-pt1-PgH2942-10.txt
finished with file  474
CREC-2017-07-27-pt1-PgH6518-24.txt
finished with file  474
CREC-2017-04-26-pt1-PgH2897-25.txt
finished with file  474
CREC-2017-04-06-pt1-PgH2791-2.txt
finished with file  474
CREC-2017-07-17-pt1-PgS4009-5.txt
finished with file  474
CREC-2017-06-15-pt1-PgH4935.txt
finished with file  475
CREC-2017-04-06-pt1-PgH2789-13.txt
finished with file  475
CREC-2017-07-24-pt1-PgH6170.txt
finished with file  479
CREC-2017-07-27-pt1-PgH6518-25.txt
finished with file  479
CREC-2017-04-27-pt1-PgH2942-11.txt
finished with file  479
CREC-2017-04-26-pt1-PgH2858.txt
finished with file  480
CREC-2017-06-28-pt1-PgH5296-6.txt
finished with file  480
CREC-2017-07-24-pt1-PgH6171.txt
finished with file  484
CREC-2017-05-08-pt1-PgH4186.txt
finished with file  484
CREC-2017-04-06-pt1-PgH2789-12.txt
finished with file 

finished with file  841
CREC-2017-05-03-pt1-PgH3309.txt
finished with file  842
CREC-2017-06-06-pt1-PgH4640-2.txt
finished with file  842
CREC-2017-06-08-pt1-PgH4715-3.txt
finished with file  843
CREC-2017-04-27-pt1-PgH2908-5.txt
finished with file  844
CREC-2017-05-18-pt1-PgH4324-5.txt
finished with file  844
CREC-2017-06-22-pt1-PgS3698-3.txt
finished with file  901
CREC-2017-04-06-pt1-PgH2768-3.txt
finished with file  902
CREC-2017-04-20-pt1-PgH2812-3.txt
finished with file  902
CREC-2017-04-25-pt1-PgH2847-13.txt
finished with file  902
CREC-2017-05-19-pt1-PgH4361-3.txt
finished with file  904
CREC-2017-06-13-pt1-PgS3459-4.txt
finished with file  904
CREC-2017-05-17-pt1-PgH4234-2.txt
finished with file  904
CREC-2017-07-28-pt1-PgH6579-12.txt
finished with file  904
CREC-2017-05-24-pt1-PgH4507-4.txt
finished with file  904
CREC-2017-04-06-pt1-PgH2792-32.txt
finished with file  904
CREC-2017-06-06-pt1-PgH4650.txt
finished with file  904
CREC-2017-07-13-pt1-PgH5772-5.txt
finished with f

CREC-2017-04-28-pt1-PgH2959-2.txt
finished with file  1175
CREC-2017-07-31-pt1-PgS4612.txt
finished with file  1177
CREC-2017-06-08-pt1-PgH4713-4.txt
finished with file  1177
CREC-2017-06-20-pt1-PgS3642-2.txt
finished with file  1178
CREC-2017-06-27-pt1-PgH5232-20.txt
finished with file  1178
CREC-2017-05-30-pt1-PgH4617-6.txt
finished with file  1178
CREC-2017-05-18-pt1-PgH4322-2.txt
finished with file  1179
CREC-2017-06-15-pt1-PgH4938-15.txt
finished with file  1179
CREC-2017-06-07-pt1-PgH4701-4.txt
finished with file  1179
CREC-2017-04-12-pt1-PgH2799-4.txt
finished with file  1179
CREC-2017-05-26-pt1-PgH4615.txt
finished with file  1179
CREC-2017-05-03-pt1-PgS2687-4.txt
finished with file  1179
CREC-2017-07-25-pt1-PgH6222-8.txt
finished with file  1179
CREC-2017-07-12-pt1-PgH5760.txt
finished with file  1179
CREC-2017-05-01-pt1-PgH2988.txt
finished with file  1179
CREC-2017-04-27-pt1-PgS2630-4.txt
finished with file  1181
CREC-2017-06-22-pt1-PgH5067-2.txt
finished with file  1181
CRE

...

used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
used existing row
used existing row
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
used

used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
used existing row
used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing ro

used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
used existing row
used existing row
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker ma

used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
no first_name
<congmember chamber="H" congress="115" role="VOTED YES">
<name type="parsed">Lujan Grisham, M.</name>
</congmember>
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
no first_name
<congmember chamber="H" congress="115" role="VOTED YES">
<name type="parsed">Lujan Grisham, M.</name>
</congmember>
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
used existing row
used existing row
us

used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker matched successfully, used existing row
speaker ma

In [92]:
next_index

8074

In [2]:
path = Path('/Users/halliday/projects/searchlight/parsing/aws_results')
speakers = pd.read_csv(path/'updatedspeakers.csv')
speakers = speakers.drop(0)

In [3]:
speakers = speakers.drop(1)

In [9]:
speakers.to_csv('allspeakers.csv', index=False)

In [5]:
allspeakers = pd.read_csv('allspeakers.csv')

In [8]:
allspeakers

,speaker_id,first_name,last_name,chamber,type,party,state,district,bio_guide_id,congress_id
0,2.0,Neil,ABERCROMBIE,HOUSE,REPRESENTATIVE,D,HI,1.0,A000014,107.0
1,1269.0,Spencer,ABRAHAM,SENATE,SENATOR,R,MI,NaN,A000355,106.0
2,2244.0,Ralph,ABRAHAM,HOUSE,REPRESENTATIVE,R,LA,5.0,A000374,114.0
3,4.0,Gary,ACKERMAN,HOUSE,REPRESENTATIVE,D,NY,5.0,A000022,107.0
4,2201.0,Alma,ADAMS,HOUSE,REPRESENTATIVE,D,NC,12.0,A000370,114.0
5,2006.0,Sandy,ADAMS,HOUSE,REPRESENTATIVE,R,FL,24.0,A000366,112.0
6,1460.0,Robert,ADERHOLT,HOUSE,REPRESENTATIVE,R,AL,4.0,A000055,113.0
7,1935.0,John,ADLER,HOUSE,REPRESENTATIVE,D,NJ,3.0,A000364,111.0
8,2229.0,Pete,AGUILAR,HOUSE,REPRESENTATIVE,D,CA,31.0,A000371,114.0
9,7.0,Daniel,AKAKA,SENATE,SENATOR,D,HI,NaN,A000069,107.0


In [10]:
def concat_frames(Path, file_names):
    count = 0
    frame_dict = {}
    for file in file_names:
        frame_dict[count] = pd.read_csv(Path/file)
        count += 1
    frames = list(frame_dict.values())
    return pd.concat(frames)

In [11]:
path = Path('/Users/halliday/projects/searchlight/parsing/aws_results/parsing_results')
file_names = ["speeches_" + str(i) + ".csv" for i in range(1, 144)]
bill_file_names = ["bills_" + str(i) + ".csv" for i in range(1,144)]
concat_speeches = concat_frames(path, file_names)
concat_bills = concat_frames(path, bill_file_names)

In [12]:
concat_speeches

,speech_id,last_name,speaker_id,proceeding_id,topic_id,word_count,speech_text,file_name,mods_file,proceeding_title,year,month,day
0,0,FORD,1334.0,proceeding_id,topic_id,127,"Mr.President, I wish to announce that the Comm...",CREC-1994-03-08-pt1-PgS46.txt,CREC-1994-03-08-pt1-PgS46.xml,NaN,1994,3,8
1,1,INOUYE,1369.0,proceeding_id,topic_id,76,"Mr.President, I would like to announce that th...",CREC-1994-03-08-pt1-PgS46.txt,CREC-1994-03-08-pt1-PgS46.xml,NaN,1994,3,8
2,2,GOSS,450.0,proceeding_id,topic_id,153,"Mr.Speaker, the White House and certain high-d...",CREC-1994-02-23-pt1-PgH28.txt,CREC-1994-02-23-pt1-PgH28.xml,NaN,1994,2,23
3,3,HATCH,1351.0,proceeding_id,topic_id,4,"Thank you, Mr. _______...",CREC-1994-03-09-pt1-PgS7.txt,CREC-1994-03-09-pt1-PgS7.xml,NaN,1994,3,9
4,4,RICHARDSON,1883.0,proceeding_id,topic_id,11,"Mr.Speaker, I rise today to ask 81 Republicans...",CREC-1994-03-16-pt1-PgH10.txt,CREC-1994-03-16-pt1-PgH10.xml,NaN,1994,3,16
5,5,LEAHY,1383.0,proceeding_id,topic_id,1361,"Mr.President, I want to speak about the former...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,NaN,1994,2,24
6,6,JOHNSTON,606.0,proceeding_id,topic_id,25,"Mr.President, what is the pending business? T...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,NaN,1994,2,24
7,7,JOHNSTON,606.0,proceeding_id,topic_id,40,"Mr.President, would it be in order to speak on...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,NaN,1994,2,24
8,8,MOAKLEY,820.0,proceeding_id,topic_id,63,"Mr.Speaker, I ask unanimous consent that the C...",CREC-1994-03-09-pt1-PgH30.txt,CREC-1994-03-09-pt1-PgH30.xml,NaN,1994,3,9
9,9,MOYNIHAN,1407.0,proceeding_id,topic_id,37,"Mr.President, I rise to announce that during t...",CREC-1994-02-22-pt1-PgS46.txt,CREC-1994-02-22-pt1-PgS46.xml,NaN,1994,2,22


In [13]:
concat_bills

,mods_file,congress_id,context,bill_number,bill_type
0,CREC-1994-01-25-pt1-PgH12.xml,103,OTHER,325,HRES
1,CREC-1994-01-25-pt1-PgH12.xml,103,HEADERLINE,325,HRES
2,CREC-1994-01-25-pt1-PgH14.xml,103,OTHER,326,HRES
3,CREC-1994-01-25-pt1-PgH14.xml,103,HEADERLINE,326,HRES
4,CREC-1994-01-25-pt1-PgH15.xml,103,HEADERLINE,327,HRES
5,CREC-1994-01-25-pt1-PgH15.xml,103,OTHER,327,HRES
6,CREC-1994-01-25-pt1-PgH16.xml,103,OTHER,329,HRES
7,CREC-1994-01-25-pt1-PgH16.xml,103,OTHER,328,HRES
8,CREC-1994-01-25-pt1-PgH16.xml,103,HEADERLINE,328,HRES
9,CREC-1994-01-25-pt1-PgH21.xml,103,HEADERLINE,197,HCONRES


In [15]:
concat_speeches.to_csv('concat_speeches.csv', index=False)
# concat_bills.to_csv('allbills.csv', index=False)

In [17]:
concat_speeches_index = pd.read_csv('concat_speeches.csv')
concat_speeches_index = concat_speeches_index.drop(['Unnamed: 0', "speech_id"], axis=1)

In [19]:
concat_speeches_index['speech_id'] = np.arange(1140685)

In [24]:
# concat_speeches_index.to_csv('allspeecheserrors.csv', index=False)
concat_speeches_index = concat_speeches_index[["speech_id", "topic_id", "word_count", "speech_text", 
                                              "file_name", "mods_file", "speaker_id", "proceeding_title",
                                              "year", "month", "day"]]
concat_speeches_index

,speech_id,topic_id,word_count,speech_text,file_name,mods_file,speaker_id,proceeding_title,year,month,day
0,0,topic_id,127,"Mr.President, I wish to announce that the Comm...",CREC-1994-03-08-pt1-PgS46.txt,CREC-1994-03-08-pt1-PgS46.xml,1334.0,NaN,1994,3,8
1,1,topic_id,76,"Mr.President, I would like to announce that th...",CREC-1994-03-08-pt1-PgS46.txt,CREC-1994-03-08-pt1-PgS46.xml,1369.0,NaN,1994,3,8
2,2,topic_id,153,"Mr.Speaker, the White House and certain high-d...",CREC-1994-02-23-pt1-PgH28.txt,CREC-1994-02-23-pt1-PgH28.xml,450.0,NaN,1994,2,23
3,3,topic_id,4,"Thank you, Mr. _______...",CREC-1994-03-09-pt1-PgS7.txt,CREC-1994-03-09-pt1-PgS7.xml,1351.0,NaN,1994,3,9
4,4,topic_id,11,"Mr.Speaker, I rise today to ask 81 Republicans...",CREC-1994-03-16-pt1-PgH10.txt,CREC-1994-03-16-pt1-PgH10.xml,1883.0,NaN,1994,3,16
5,5,topic_id,1361,"Mr.President, I want to speak about the former...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,1383.0,NaN,1994,2,24
6,6,topic_id,25,"Mr.President, what is the pending business? T...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,606.0,NaN,1994,2,24
7,7,topic_id,40,"Mr.President, would it be in order to speak on...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,606.0,NaN,1994,2,24
8,8,topic_id,63,"Mr.Speaker, I ask unanimous consent that the C...",CREC-1994-03-09-pt1-PgH30.txt,CREC-1994-03-09-pt1-PgH30.xml,820.0,NaN,1994,3,9
9,9,topic_id,37,"Mr.President, I rise to announce that during t...",CREC-1994-02-22-pt1-PgS46.txt,CREC-1994-02-22-pt1-PgS46.xml,1407.0,NaN,1994,2,22


In [25]:
concat_speeches_index.to_csv('allspeeches.csv', index=False)

In [76]:
allspeakers = pd.read_csv("/Users/halliday/projects/searchlight/parsing/aws_results/updatedspeakers.csv")
allspeakers

In [86]:
allspeakers.to_csv("allspeakers.csv", index=False)

In [88]:
allspeakers.dtypes

speaker_id      float64
first_name       object
last_name        object
chamber          object
type             object
party            object
state            object
district        float64
bio_guide_id     object
congress_id     float64
dtype: object

# Title Fixing

In [26]:
# path = Path('/Users/halliday/projects/searchlight/parsing/')
speeches = pd.read_csv('allspeeches.csv')
speeches

,speech_id,topic_id,word_count,speech_text,file_name,mods_file,speaker_id,proceeding_title,year,month,day
0,0,topic_id,127,"Mr.President, I wish to announce that the Comm...",CREC-1994-03-08-pt1-PgS46.txt,CREC-1994-03-08-pt1-PgS46.xml,1334.0,NaN,1994,3,8
1,1,topic_id,76,"Mr.President, I would like to announce that th...",CREC-1994-03-08-pt1-PgS46.txt,CREC-1994-03-08-pt1-PgS46.xml,1369.0,NaN,1994,3,8
2,2,topic_id,153,"Mr.Speaker, the White House and certain high-d...",CREC-1994-02-23-pt1-PgH28.txt,CREC-1994-02-23-pt1-PgH28.xml,450.0,NaN,1994,2,23
3,3,topic_id,4,"Thank you, Mr. _______...",CREC-1994-03-09-pt1-PgS7.txt,CREC-1994-03-09-pt1-PgS7.xml,1351.0,NaN,1994,3,9
4,4,topic_id,11,"Mr.Speaker, I rise today to ask 81 Republicans...",CREC-1994-03-16-pt1-PgH10.txt,CREC-1994-03-16-pt1-PgH10.xml,1883.0,NaN,1994,3,16
5,5,topic_id,1361,"Mr.President, I want to speak about the former...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,1383.0,NaN,1994,2,24
6,6,topic_id,25,"Mr.President, what is the pending business? T...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,606.0,NaN,1994,2,24
7,7,topic_id,40,"Mr.President, would it be in order to speak on...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,606.0,NaN,1994,2,24
8,8,topic_id,63,"Mr.Speaker, I ask unanimous consent that the C...",CREC-1994-03-09-pt1-PgH30.txt,CREC-1994-03-09-pt1-PgH30.xml,820.0,NaN,1994,3,9
9,9,topic_id,37,"Mr.President, I rise to announce that during t...",CREC-1994-02-22-pt1-PgS46.txt,CREC-1994-02-22-pt1-PgS46.xml,1407.0,NaN,1994,2,22


In [58]:
speeches = speeches.drop(['Unnamed: 0', 'speech_id'], axis=1)

In [59]:
ids = np.arange(557818)

In [60]:
speeches['speech_id'] = ids

In [61]:
speeches = speeches[['speech_id', 'topic_id', 'word_count', 'speech_text', 'file_name', 'mods_file', 'last_name', 'speaker_id', 'proceeding_title', 'year', 'month', 'day']]

In [62]:
allspeeches = speeches.drop(['last_name'], axis=1)
allspeeches

,speech_id,topic_id,word_count,speech_text,file_name,mods_file,speaker_id,proceeding_title,year,month,day
0,0,topic_id,153,"Mr.Speaker, the White House and certain high-d...",CREC-1994-02-23-pt1-PgH28.txt,CREC-1994-02-23-pt1-PgH28.xml,450.0,NaN,1994,2,23
1,1,topic_id,11,"Mr.Speaker, I rise today to ask 81 Republicans...",CREC-1994-03-16-pt1-PgH10.txt,CREC-1994-03-16-pt1-PgH10.xml,1883.0,NaN,1994,3,16
2,2,topic_id,341,"Mr.Speaker, I move to suspend the rules and ag...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,850.0,NaN,1994,4,26
3,3,topic_id,170,"Mr.Speaker, I yield myself such time as I may ...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,850.0,NaN,1994,4,26
4,4,topic_id,350,"Mr.Speaker, I yield myself such time as I may ...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,322.0,NaN,1994,4,26
5,5,topic_id,117,"Mr.Speaker, as the author of House Concurrent ...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,1166.0,NaN,1994,4,26
6,6,topic_id,9,"Mr.Speaker, I yield back the balance of my tim...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,322.0,NaN,1994,4,26
7,7,topic_id,76,"Mr.Speaker, I, too, yield back the balance of ...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,850.0,NaN,1994,4,26
8,8,topic_id,314,"Mr.Speaker, the debate on the crime bill begin...",CREC-1994-04-12-pt1-PgH21.txt,CREC-1994-04-12-pt1-PgH21.xml,1036.0,NaN,1994,4,12
9,9,topic_id,63,"Mr.Speaker, I ask unanimous consent that the C...",CREC-1994-03-09-pt1-PgH30.txt,CREC-1994-03-09-pt1-PgH30.xml,820.0,NaN,1994,3,9


In [72]:
allspeeches[allspeeches['proceeding_title'].isnull()]

,speech_id,topic_id,word_count,speech_text,file_name,mods_file,speaker_id,proceeding_title,year,month,day


In [54]:
# allspeeches = pd.read_csv(path/'parsing_results/allspeeches.csv')
allspeeches = allspeeches.drop(['Unnamed: 0'], axis=1)
allspeeches.to_csv(path/'parsing_results/allspeeches.csv', index=False)

In [2]:
path = Path('/Users/halliday/projects/ltd/ltd-congressscraper/searchlight-website/class/csv')
allspeeches = pd.read_csv(path/'allspeeches.csv')
allspeeches

,speech_id,topic_id,word_count,speech_text,file_name,mods_file,speaker_id,proceeding_title,year,month,day
0,0,topic_id,153,"Mr.Speaker, the White House and certain high-d...",CREC-1994-02-23-pt1-PgH28.txt,CREC-1994-02-23-pt1-PgH28.xml,450.0,NaN,1994,2,23
1,1,topic_id,11,"Mr.Speaker, I rise today to ask 81 Republicans...",CREC-1994-03-16-pt1-PgH10.txt,CREC-1994-03-16-pt1-PgH10.xml,1883.0,NaN,1994,3,16
2,2,topic_id,341,"Mr.Speaker, I move to suspend the rules and ag...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,850.0,NaN,1994,4,26
3,3,topic_id,170,"Mr.Speaker, I yield myself such time as I may ...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,850.0,NaN,1994,4,26
4,4,topic_id,350,"Mr.Speaker, I yield myself such time as I may ...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,322.0,NaN,1994,4,26
5,5,topic_id,117,"Mr.Speaker, as the author of House Concurrent ...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,1166.0,NaN,1994,4,26
6,6,topic_id,9,"Mr.Speaker, I yield back the balance of my tim...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,322.0,NaN,1994,4,26
7,7,topic_id,76,"Mr.Speaker, I, too, yield back the balance of ...",CREC-1994-04-26-pt1-PgH38.txt,CREC-1994-04-26-pt1-PgH38.xml,850.0,NaN,1994,4,26
8,8,topic_id,314,"Mr.Speaker, the debate on the crime bill begin...",CREC-1994-04-12-pt1-PgH21.txt,CREC-1994-04-12-pt1-PgH21.xml,1036.0,NaN,1994,4,12
9,9,topic_id,63,"Mr.Speaker, I ask unanimous consent that the C...",CREC-1994-03-09-pt1-PgH30.txt,CREC-1994-03-09-pt1-PgH30.xml,820.0,NaN,1994,3,9


In [68]:
month_dict = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}
for i in range(1, 1140685):
    if type(speeches.loc[i, 'month']) == int:
        if i % 10000 == 0:
            print(i)
        speeches.loc[i, 'month'] = month_dict[speeches.loc[i, 'month']]        

960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000


In [73]:
speeches.loc[1140684, 'month']

'May'

In [74]:
speeches

,speech_id,topic_id,word_count,speech_text,file_name,mods_file,speaker_id,proceeding_title,year,month,day
0,0,topic_id,127,"Mr.President, I wish to announce that the Comm...",CREC-1994-03-08-pt1-PgS46.txt,CREC-1994-03-08-pt1-PgS46.xml,1334.0,NaN,1994,March,8
1,1,topic_id,76,"Mr.President, I would like to announce that th...",CREC-1994-03-08-pt1-PgS46.txt,CREC-1994-03-08-pt1-PgS46.xml,1369.0,NaN,1994,March,8
2,2,topic_id,153,"Mr.Speaker, the White House and certain high-d...",CREC-1994-02-23-pt1-PgH28.txt,CREC-1994-02-23-pt1-PgH28.xml,450.0,NaN,1994,February,23
3,3,topic_id,4,"Thank you, Mr. _______...",CREC-1994-03-09-pt1-PgS7.txt,CREC-1994-03-09-pt1-PgS7.xml,1351.0,NaN,1994,March,9
4,4,topic_id,11,"Mr.Speaker, I rise today to ask 81 Republicans...",CREC-1994-03-16-pt1-PgH10.txt,CREC-1994-03-16-pt1-PgH10.xml,1883.0,NaN,1994,March,16
5,5,topic_id,1361,"Mr.President, I want to speak about the former...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,1383.0,NaN,1994,February,24
6,6,topic_id,25,"Mr.President, what is the pending business? T...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,606.0,NaN,1994,February,24
7,7,topic_id,40,"Mr.President, would it be in order to speak on...",CREC-1994-02-24-pt1-PgS10.txt,CREC-1994-02-24-pt1-PgS10.xml,606.0,NaN,1994,February,24
8,8,topic_id,63,"Mr.Speaker, I ask unanimous consent that the C...",CREC-1994-03-09-pt1-PgH30.txt,CREC-1994-03-09-pt1-PgH30.xml,820.0,NaN,1994,March,9
9,9,topic_id,37,"Mr.President, I rise to announce that during t...",CREC-1994-02-22-pt1-PgS46.txt,CREC-1994-02-22-pt1-PgS46.xml,1407.0,NaN,1994,February,22


In [89]:
speeches.to_csv('allspeeches.csv', index=False)

# Double Checking

In [68]:
speechesDir = Path('/Users/halliday/projects/searchlight/parsing/aws_setup/parsing_results')
speeches = pd.read_csv(speechesDir/'concat_test.csv')
speeches[speeches['last_name'] == '']

,speech_id,last_name,speaker_id,proceeding_id,topic_id,word_count,speech_text,file_name,mods_file,proceeding_title,year,month,day


In [49]:
speakersDir = Path('/Users/halliday/projects/searchlight/parsing/aws_setup')
speakers = pd.read_csv(speakersDir/'updatedspeakers.csv')
speakers[speakers['last_name'] == 'UDALL']

,speaker_id,first_name,last_name,chamber,type,party,state,district,bio_guide_id,congress_id
1159,1567.0,Tom,UDALL,SENATE,SENATOR,D,NM,NaN,U000039,113.0
1160,1595.0,Mark,UDALL,SENATE,SENATOR,D,CO,NaN,U000038,113.0


In [30]:
speeches.dtypes

speech_id             int64
last_name            object
speaker_id          float64
proceeding_id        object
topic_id             object
word_count            int64
speech_text          object
file_name            object
mods_file            object
proceeding_title     object
year                  int64
month                 int64
day                   int64
dtype: object

In [95]:
billsDir = Path('/Users/halliday/projects/searchlight/parsing')
bills = pd.read_csv(billsDir/'bills_1.csv')
bills

,mods_file,congress_id,context,bill_number,bill_type
0,CREC-2017-04-06-pt1-PgH2759-2.xml,115,FIRSTPARAGRAPH,242,HRES
1,CREC-2017-04-06-pt1-PgH2759-2.xml,115,OTHER,444,S
2,CREC-2017-04-06-pt1-PgH2759-2.xml,115,HEADERLINE,1219,HR
3,CREC-2017-04-06-pt1-PgH2759-2.xml,115,FIRSTPARAGRAPH,1219,HR
4,CREC-2017-04-06-pt1-PgH2759-2.xml,115,OTHER,4854,HR
5,CREC-2017-04-06-pt1-PgH2759-2.xml,115,OTHER,4852,HR
6,CREC-2017-04-06-pt1-PgH2759-2.xml,115,OTHER,1219,HR
7,CREC-2017-04-06-pt1-PgH2759-2.xml,115,OTHER,4855,HR
8,CREC-2017-04-06-pt1-PgH2759-2.xml,115,OTHER,242,HRES
9,CREC-2017-04-06-pt1-PgH2767-3.xml,115,TITLE,246,HRES


In [77]:
speakers = pd.read_csv(speechesDir/'updatedspeakers.csv')
speakers[speakers['last_name'] == 'MERKLEY']

,speaker_id,first_name,last_name,chamber,type,party,state,district,bio_guide_id,congress_id
789,1900.0,Jeff,MERKLEY,SENATE,SENATOR,D,OR,NaN,M001176,113.0


In [438]:
speakers[speakers['last_name'] == 'BACON']

,speaker_id,first_name,last_name,chamber,type,party,state,district,bio_guide_id,congress_id
32,2337,Don,BACON,HOUSE,NaN,R,NE,NaN,B001298,115


In [439]:
speeches.dtypes

speech_id        object
last_name        object
speaker_id       object
proceeding_id    object
topic_id         object
word_count       object
speech_text      object
file_name        object
mods_file        object
dtype: object

In [440]:
speakers.dtypes

speaker_id        int64
first_name       object
last_name        object
chamber          object
type             object
party            object
state            object
district        float64
bio_guide_id     object
congress_id       int64
dtype: object

In [463]:
df = pd.DataFrame(columns=['a', 'b'])

In [466]:
df = df.append({'a': 0, 'b': 1}, ignore_index=True)

In [469]:
df['c'] = [0 for i in range(3)]

In [471]:
df['d'], df['e'] = 1, 2

In [475]:
df.loc[0,'a'] = 4

In [476]:
df

,a,b,c,d,e
0,4,1,0,1,2
1,0,1,0,1,2
2,0,1,0,1,2


In [6]:
x = {'a': 1}
y = dict(x)
y['b'] = 2
print(x)
print(y)

{'a': 1}
{'a': 1, 'b': 2}


# Type Fixing

In [8]:
speakers = pd.read_csv('/Users/halliday/projects/searchlight/parsing/aws_parsing_setup/updatedspeakers.csv')
speakers.dtypes

speaker_id      float64
first_name       object
last_name        object
chamber          object
type             object
party            object
state            object
district        float64
bio_guide_id     object
congress_id     float64
dtype: object

In [11]:
speakers['speaker_id'] = speakers['speaker_id'].astype('float64')
speakers.dtypes

speaker_id      float64
first_name       object
last_name        object
chamber          object
type             object
party            object
state            object
district        float64
bio_guide_id     object
congress_id     float64
dtype: object